Simple word2vec model with 200 features for each utterance, not considering other features (speaker etc.)

Doc2Vec model for all chats

In [61]:
import os
import json
import gensim

# path = r"C:\Users\user\git\NLPProj\Frames-dataset\chats"


def build_label_dict():
    d = {}
    label_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/labels.txt"
    label_file = open(label_path, encoding='utf-8')
    for line in label_file:
        k, v = line.split("\t")
        d[k] = v.strip()
    return d

def gen_chat_model(label_dict):
    chat_path = r"/home/amirgilad/Documents/git/NLPProj/Frames-dataset/chats"
    chats = []
    for filename in os.listdir(chat_path):
        chat_file = open(os.path.join(chat_path, filename), encoding='utf-8')
        chat = json.load(chat_file)
        sents = [] # chat level docs
        if 'turns' in chat:
            for turn in chat['turns']:
                sents.extend( turn['text'].split() )
        tagged_chat = gensim.models.doc2vec.TaggedDocument(sents, label_dict[filename])
        chats.append( tagged_chat )
    return gensim.models.Doc2Vec(chats, size=100, window=8, min_count=1, workers=4)

d = build_label_dict()
chat_model = gen_chat_model(d)


[('1', 0.1597049981355667), ('0', 0.028184477239847183)]

In [64]:
dv = chat_model.infer_vector("I would like to order".split())
chat_model.docvecs.most_similar(positive=[dv], topn=2)

[('1', -0.14040948450565338), ('0', -0.14046819508075714)]

Define the RNN - TBD

In [96]:
import numpy as np
import sys
# should change to relative import
sys.path.insert(0, '/home/amirgilad/Documents/git/NLPProj/kerasAttention')
import attention_dense
import attention_lstm

input_dims = 32

inputs = np.array([i for i in range(input_dims)])
attention_probs = Dense(input_dims, activation='softmax', name='attention_probs')(inputs)
attention_mul = merge([inputs, attention_probs], output_shape=input_dims, name='attention_mul', mode='mul')


ValueError: Layer attention_probs was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])]. All inputs to the layer should be tensors.